In [1]:
import sys
import importlib
import numpy as np
from saxstats import saxstats
from Ligand import Ligand
from Geometry import *
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import time
from array import array
from scipy.optimize import minimize
from spyrmsd import rmsd as srmsd

In [3]:
lig_gt = Ligand('1FIN_ligand.sdf')

This ligand does not seem to have hydrogens modeled into them
Consider doing so for best X-ray scattering signal accuracy
Atom 14 is qualified to have an improper term
Atom 16 is qualified to have an improper term
Atom 18 is qualified to have an improper term
Atom 20 is qualified to have an improper term
Processing candidate [14 13 15 16]
3
Graph is split to 3 subgraphs
Processing candidate [16 14 17 18]
3
Graph is split to 3 subgraphs
Processing candidate [18 16 19 20]
3
Graph is split to 3 subgraphs
Processing candidate [20 15 18 21]
3
Graph is split to 3 subgraphs
[[array([14, 13, 15, 16]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])], [array([20, 21, 15, 18]), array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30])]]


[12:10:38] Molecule does not have explicit Hs. Consider calling AddHs()


In [4]:
sp = {'i': np.array([0, 30])}

In [5]:
lig_gt.set_coordinates(lig_gt.transform(0, sp))
lig_gt.save('test_improper.pdb')

[[-11.061  204.333  118.224 ]
 [-10.692  205.181  116.99  ]
 [-11.1125 205.65   116.282 ]]
[[-11.399 205.978 113.562]
 [-10.603 205.789 114.77 ]
 [-10.873 205.892 115.595]]


In [7]:
sp = {'a': np.zeros(lig_gt.num_angle), 't': np.mean(lig_gt.get_coordinates(), 0)}
sp['a'][0] = 30

In [5]:
lig_gt.agroup

[[array([0, 7, 4]), array([0, 1, 2, 3])],
 [array([ 4, 11,  8]), array([0, 1, 2, 3, 4, 5, 6, 7])],
 [array([ 7,  4, 11]), array([0, 1, 2, 3, 7])],
 [array([ 8, 12, 13]),
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])],
 [array([11,  8, 12]), array([ 0,  1,  2,  3,  4,  5,  6,  7, 11])],
 [array([12, 13, 14]),
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])]]

In [8]:
lig_gt.set_coordinates(lig_gt.transform(0, sp))

In [7]:
lig_gt.save('test_angle.pdb')

In [2]:
lig_gt = Ligand('1FIN_ligand.sdf')

This ligand does not seem to have hydrogens modeled into them
Consider doing so for best X-ray scattering signal accuracy
[[array([14, 13, 15, 16]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])], [array([16, 17, 14, 18]), array([17])], [array([18, 19, 16, 20]), array([19])], [array([20, 21, 15, 18]), array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30])]]


[12:47:16] Molecule does not have explicit Hs. Consider calling AddHs()


In [3]:
def x_to_dict(x, lig):
    return {'t': x[:3], 'r': x[3:6]*18, 'a': x[6:6+lig.num_angle], 
            'i': x[6+lig.num_angle:6+lig.num_angle+lig.num_improper],
            'd': x[6+lig.num_angle+lig.num_improper:]*18}
def myminfunc(x, lig, lig_coord_gt):
    x_dict = x_to_dict(x, lig)
    return pureRMSD(lig.transform(0, x_dict), lig_coord_gt)
def mysymmminfunc(x, lig, lig_coord_gt, isomorphism=None, return_isomorphism=False):
    x_dict = x_to_dict(x, lig)
    if return_isomorphism:
        return symmRMSD(lig.transform(0, x_dict), lig_coord_gt, lig.A, lig.G, isomorphism)
    else:
        return symmRMSD(lig.transform(0, x_dict), lig_coord_gt, lig.A, lig.G, isomorphism)[0]

In [4]:
sp_np = {'t': np.random.random(3) * 180,
         'r': np.random.random(3) * 180,
         'a': np.random.random(lig_gt.num_angle) * 10,
         'd': np.random.random(lig_gt.num_torsion) * 180}

In [4]:
# lig_gt.set_coordinates(lig_gt.transform(0, sp))
lig_coord_gt = lig_gt.get_coordinates()

In [5]:
lig = Ligand('1FIN_ligand.sdf')
lig.generate_conformers(1)
AllChem.MMFFOptimizeMolecule(lig.molecule)
# lig.get_coordinates()
# lig.set_coordinates(lig.transform(0, sp_np))

This ligand does not seem to have hydrogens modeled into them
Consider doing so for best X-ray scattering signal accuracy
[[array([14, 13, 15, 16]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])], [array([16, 17, 14, 18]), array([17])], [array([18, 19, 16, 20]), array([19])], [array([20, 21, 15, 18]), array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30])]]
Generated 1 conformers


[12:47:30] Molecule does not have explicit Hs. Consider calling AddHs()
[12:47:30] Molecule does not have explicit Hs. Consider calling AddHs()
[12:47:30] Molecule does not have explicit Hs. Consider calling AddHs()


1

In [6]:
res = minimize(myminfunc, np.zeros(6+lig.num_angle+lig.num_torsion+lig.num_improper), 
               args=(lig, lig_coord_gt), 
               method='L-BFGS-B', 
               options={'iprint':50, 'maxfun': 100000, 'gtol': 1e-4})
               #bounds=[(None, None), (None, None), (None, None), (0, 180), (0, 360), 
                       #*([(None, None)] * lig.num_torsion)], options={'iprint': 10})

print(res)
xdict = x_to_dict(res.x, lig)
for x in xdict.keys():
    print(x, xdict[x])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           22     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.36637D+02    |proj g|=  8.68440D-01


 This problem is unconstrained.



At iterate   50    f=  6.19620D-01    |proj g|=  4.30038D-02

At iterate  100    f=  4.61594D-01    |proj g|=  8.21603D-03

At iterate  150    f=  3.85626D-01    |proj g|=  2.11790D-02

At iterate  200    f=  3.73949D-01    |proj g|=  4.87402D-02

At iterate  250    f=  3.65450D-01    |proj g|=  1.92658D-03

At iterate  300    f=  3.63626D-01    |proj g|=  1.85860D-03

At iterate  350    f=  3.63111D-01    |proj g|=  4.83404D-03

At iterate  400    f=  3.60775D-01    |proj g|=  1.07851D-02

At iterate  450    f=  3.60041D-01    |proj g|=  8.54149D-03

At iterate  500    f=  3.59341D-01    |proj g|=  1.19650D-03

At iterate  550    f=  3.57832D-01    |proj g|=  1.93748D-03

At iterate  600    f=  3.57557D-01    |proj g|=  3.18465D-03

At iterate  650    f=  3.57239D-01    |proj g|=  6.83076D-04

At iterate  700    f=  3.57069D-01    |proj g|=  1.21470D-03

At iterate  750    f=  3.56843D-01    |proj g|=  2.94755D-03

At iterate  800    f=  3.56814D-01    |proj g|=  2.13317D-03

At iter

In [6]:
iso=None

In [7]:
rmsd, iso = mysymmminfunc(np.zeros(6+lig.num_angle+lig.num_torsion+lig.num_improper),
                          lig, lig_coord_gt, isomorphism=None, return_isomorphism=True)
print(rmsd)

236.62853791438704


In [27]:
# # Run with bounds on angles
# res = minimize(mysymmminfunc, np.zeros(6+lig.num_angle+lig.num_torsion), 
#                args=(lig, lig_coord_gt, iso), method='L-BFGS-B', 
#                options={'iprint':50, 'maxfun': 100000, 'gtol': 1e-4},
#                bounds=[(None, None)] * 6 + [(-0.01, 0.01)] * lig.num_angle + [(None, None)] * lig.num_torsion)
# #                bounds=[(None, None), (None, None), (None, None), (0, 180), (0, 360), 
#                        #*([(None, None)] * lig.num_torsion)], options={'iprint': 10})

# print(res)
# xdict = x_to_dict(res.x, lig)
# for x in xdict.keys():
#     print(x, xdict[x])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           20     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.36637D+02    |proj g|=  8.68440D-01
  ys=-3.101E-06  -gs= 9.997E-01 BFGS update SKIPPED
  ys=-1.428E-03  -gs= 2.199E+01 BFGS update SKIPPED
  ys=-2.945E-04  -gs= 8.973E+00 BFGS update SKIPPED
  ys=-1.188E-03  -gs= 1.889E+01 BFGS update SKIPPED
  ys=-1.104E-03  -gs= 2.979E+01 BFGS update SKIPPED

At iterate   50    f=  8.99814D-01    |proj g|=  5.10193D-02

At iterate  100    f=  8.04711D-01    |proj g|=  2.68618D-04
      fun: 0.8047109812315043
 hess_inv: <20x20 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 8.92619238e-06, -1.99173853e-05,  1.19238028e-05,  5.45563598e-05,
        2.00950369e-05, -9.02611325e-06, -2.39486209e-04, -3.07958103e-03,
        6.85829171e-04,  1.54080082e-03,  8.65818528e-04, -1.08908438e-03,
        4.49640288e-06, -1.01141318e-05,  1.95177209e-05, -2.09

In [8]:

sp0 = np.concatenate([lig_gt.get_coordinates(0).mean(0), np.zeros(3 + lig.num_angle+lig.num_torsion+lig.num_improper)])
spb = [(x-5, x+5) for x in sp0[:3]] + [(-20, 20)] * 3 + [(-20, 20)] * lig.num_angle + \
       [(-20, 20)] * lig.num_improper + [(-20, 20)] * lig.num_torsion
res = minimize(mysymmminfunc, np.zeros(6+lig.num_angle+lig.num_torsion+lig.num_improper), 
               args=(lig, lig_coord_gt, iso), method='L-BFGS-B', 
               options={'iprint':50, 'maxfun': 100000},
               bounds=spb
              )
               #bounds=[(None, None), (None, None), (None, None), (0, 180), (0, 360), 
                       #*([(None, None)] * lig.num_torsion)], options={'iprint': 10})

print(res)
xdict = x_to_dict(res.x, lig)
for x in xdict.keys():
    print(x, xdict[x])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           24     M =           10

At X0         3 variables are exactly at the bounds

At iterate    0    f=  1.13245D+01    |proj g|=  4.41521D-01
  ys=-3.225E-02  -gs= 9.355E-01 BFGS update SKIPPED

At iterate   50    f=  7.11242D-01    |proj g|=  3.23140D-02

At iterate  100    f=  5.89880D-01    |proj g|=  1.10806D-02

At iterate  150    f=  5.22579D-01    |proj g|=  1.21846D-02

At iterate  200    f=  4.97719D-01    |proj g|=  3.43390D-03

At iterate  250    f=  4.96106D-01    |proj g|=  2.98743D-02

At iterate  300    f=  4.91692D-01    |proj g|=  1.56350D-03

At iterate  350    f=  4.90554D-01    |proj g|=  6.96013D-03

At iterate  400    f=  4.90317D-01    |proj g|=  7.50761D-04

At iterate  450    f=  4.90195D-01    |proj g|=  6.77225D-04

At iterate  500    f=  4.90159D-01    |proj g|=  4.06924D-04

At iterate  550    f=  4.90135D-01    |proj g|=  3.64975D-04

At iterate  600    f=  4.90116D-01  

In [9]:
lig.set_coordinates(lig.transform(0, x_to_dict(res.x, lig)))
lig.save('test_min_improper.pdb')

In [47]:
# print("Ground truth")
# print(lig_coord_gt)

# print("Minimized result")
# print(lig.transform(0, res.x))

print("Initial RMSD:", end=' ')
print(pureRMSD(lig_coord_gt, lig.get_coordinates()))

print("Initial Kabsch RMSD:", end=' ')
print(Kabsch_RMSD(lig_coord_gt, lig.get_coordinates())[0])

print("Final RMSD:", end=' ')
print(pureRMSD(lig_coord_gt, lig.transform(0, x_to_dict(res.x, lig))))

print("Final Kabsch RMSD:", end=' ')
print(Kabsch_RMSD(lig_coord_gt, lig.transform(0, x_to_dict(res.x, lig)))[0])

print("Final symmetry aligned RMSD: ", end=' ')
print(symmRMSD(lig.transform(0, x_to_dict(res.x, lig)), lig_coord_gt, lig.A, lig.G, iso)[0])

Initial RMSD: 236.69560425048505
Initial Kabsch RMSD: 2.401188689199162
Final RMSD: 0.8038656097057654
Final Kabsch RMSD: 0.8038529702847935
Final symmetry aligned RMSD:  0.4555604277957224


In [4]:
mol = Chem.MolFromMolFile('1FIN_ligand.sdf')

In [5]:
dir(mol.GetAtomWithIdx(1))

['ClearProp',
 'DescribeQuery',
 'GetAtomMapNum',
 'GetAtomicNum',
 'GetBonds',
 'GetBoolProp',
 'GetChiralTag',
 'GetDegree',
 'GetDoubleProp',
 'GetExplicitBitVectProp',
 'GetExplicitValence',
 'GetFormalCharge',
 'GetHybridization',
 'GetIdx',
 'GetImplicitValence',
 'GetIntProp',
 'GetIsAromatic',
 'GetIsotope',
 'GetMass',
 'GetMonomerInfo',
 'GetNeighbors',
 'GetNoImplicit',
 'GetNumExplicitHs',
 'GetNumImplicitHs',
 'GetNumRadicalElectrons',
 'GetOwningMol',
 'GetPDBResidueInfo',
 'GetProp',
 'GetPropNames',
 'GetPropsAsDict',
 'GetQueryType',
 'GetSmarts',
 'GetSymbol',
 'GetTotalDegree',
 'GetTotalNumHs',
 'GetTotalValence',
 'GetUnsignedProp',
 'HasOwningMol',
 'HasProp',
 'HasQuery',
 'InvertChirality',
 'IsInRing',
 'IsInRingSize',
 'Match',
 'NeedsUpdatePropertyCache',
 'SetAtomMapNum',
 'SetAtomicNum',
 'SetBoolProp',
 'SetChiralTag',
 'SetDoubleProp',
 'SetExplicitBitVectProp',
 'SetFormalCharge',
 'SetHybridization',
 'SetIntProp',
 'SetIsAromatic',
 'SetIsotope',
 'Set

In [6]:
[(x, 
  mol.GetAtomWithIdx(x).GetSymbol(), 
  mol.GetAtomWithIdx(x).GetHybridization(), 
  mol.GetAtomWithIdx(x).GetDegree(), 
  mol.GetAtomWithIdx(x).GetHybridization() == rdkit.Chem.rdchem.HybridizationType.SP3 ) for x in range(mol.GetNumAtoms())]




[(0, 'P', rdkit.Chem.rdchem.HybridizationType.SP3, 4, True),
 (1, 'O', rdkit.Chem.rdchem.HybridizationType.SP2, 1, False),
 (2, 'O', rdkit.Chem.rdchem.HybridizationType.SP3, 1, True),
 (3, 'O', rdkit.Chem.rdchem.HybridizationType.SP3, 1, True),
 (4, 'P', rdkit.Chem.rdchem.HybridizationType.SP3, 4, True),
 (5, 'O', rdkit.Chem.rdchem.HybridizationType.SP2, 1, False),
 (6, 'O', rdkit.Chem.rdchem.HybridizationType.SP3, 1, True),
 (7, 'O', rdkit.Chem.rdchem.HybridizationType.SP3, 2, True),
 (8, 'P', rdkit.Chem.rdchem.HybridizationType.SP3, 4, True),
 (9, 'O', rdkit.Chem.rdchem.HybridizationType.SP2, 1, False),
 (10, 'O', rdkit.Chem.rdchem.HybridizationType.SP3, 1, True),
 (11, 'O', rdkit.Chem.rdchem.HybridizationType.SP3, 2, True),
 (12, 'O', rdkit.Chem.rdchem.HybridizationType.SP3, 2, True),
 (13, 'C', rdkit.Chem.rdchem.HybridizationType.SP3, 2, True),
 (14, 'C', rdkit.Chem.rdchem.HybridizationType.SP3, 3, True),
 (15, 'O', rdkit.Chem.rdchem.HybridizationType.SP3, 2, True),
 (16, 'C', rdki

In [7]:
mol.GetAtomWithIdx(14).GetDegree()

3

In [15]:
mol.GetAtomWithIdx(14).GetBonds()[2].GetEndAtomIdx()

16